In [2]:
## Demo
import getout_of_text_3 as got3

/Users/ejacquot/Documents/Github/getout_of_text_3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
coca_corpus = got3.read_corpus('../../data/english-corpora.org/coca/')

COCA Genres:   0%|          | 0/8 [00:00<?, ?genre/s]

Processing genre: mag


COCA Genres:  12%|█▎        | 1/8 [00:00<00:06,  1.00genre/s]

Finished genre: mag (total files: 30)
Processing genre: web


COCA Genres:  25%|██▌       | 2/8 [00:01<00:05,  1.02genre/s]

...

Finished genre: blog (total files: 34)
Processing genre: fic


COCA Genres:  88%|████████▊ | 7/8 [00:07<00:01,  1.12s/genre]

Finished genre: fic (total files: 30)
Processing genre: tvm


COCA Genres: 100%|██████████| 8/8 [00:08<00:00,  1.07s/genre]

Finished genre: tvm (total files: 30)


In [4]:
# use time elapse to show query times. multiprocessing is available for faster searches.
import pandas as pd
before = pd.Timestamp.now()

results = got3.search_keyword_corpus('best system', coca_corpus, 
                                            case_sensitive=False,
                                            show_context=True, 
                                            context_words=15,
                                            output='json',
                                            parallel=True)
after = pd.Timestamp.now()
print('time elapsed:', after - before)

time elapsed: 0 days 00:00:24.951312


In [7]:
legal_question = "Could the EPA's schema of generation shifting (lowering national coal power production by 11% over 15 years) constitute a best system given our ordinary meaning of the phrase?"

In [8]:
### Trying it on got3
import getout_of_text_3 as got3

# First, perform a keyword search to get context data
# Use the new got3.embedding.gemma function with search results
result = got3.embedding.gemma.task(
    #statutory_language="The term “standard of performance” means a standard for emissions of air pollutants which reflects the degree of emission limitation achievable through the application of the best system of emission reduction which (taking into account the cost of achieving such reduction and any nonair quality health and environmental impact and energy requirements) the Administrator determines has been adequately demonstrated.",
    statutory_language=legal_question,
    ambiguous_term="best system",
    year_enacted=2015,
    search_results=results, # Pass the JSON results from search_keyword_corpus
    model="google/embeddinggemma-300m"
)
print('')

print("🎯 Top 10 most relevant contexts:")
for i, item in enumerate(result['all_ranked'][:10]):
    print(f"{i+1}. Genre: {item['genre']}, Score: {item['score']:.4f}")
    print(f"   Context: {item['context'][:100]}...")
    print()


📚 Using pre-computed search results for 'best system'
📚 Found 150 context examples across 248 genres
🤖 Loading model: google/embeddinggemma-300m

🎯 RESULTS:

Most relevant context from news_2018 (score: 0.4480)
Context: officials with devising plans to achieve them , it will define what constitutes the " **best system** of emission reduction " that utilities can undertake with technology that has been demonstrated to

🎯 Top 10 most relevant contexts:
1. Genre: news_2018, Score: 0.4480
   Context: officials with devising plans to achieve them , it will define what constitutes the " **best system*...

2. Genre: news_2005, Score: 0.3856
   Context: age 23 or 24 ? It makes no sense to me . It 's the **best system** I know to be fair to people . It ...

3. Genre: acad_1993, Score: 0.3822
   Context: to those for submitting implementation plans under section 110 . These EPA guidelines specified the ...

4. Genre: blog_08, Score: 0.3565
   Context: air pollutants which reflects the degree of 

## Thinking through integration on the app

In [25]:
#import sentence_transformers
# Load the model
from sentence_transformers import SentenceTransformer
import torch

model = "google/embeddinggemma-300m"

embedding_model = SentenceTransformer(model)

query = "should we read the phrase 'best system' narrowly or broadly, in ordinary meaning?"

# Convert search results to documents for embedding
documents = []
document_metadata = []

for genre, texts in results.items():
    for text_id, context in texts.items():
        # Format as title | text for better embedding performance
        doc = f"title: {genre.upper()} Text {text_id} | text: {context}"
        documents.append(doc)
        document_metadata.append({
            'genre': genre,
            'text_id': text_id,
            'context': context
        })

In [26]:
# Encode query and documents
query_prompt = f"task: question answering | query: {query}"
query_embedding = embedding_model.encode_query(query_prompt)
document_embeddings = embedding_model.encode_document(documents)

# Calculate similarities
similarities = embedding_model.similarity(query_embedding, document_embeddings)

# Get rankings
ranked_indices = torch.argsort(similarities, descending=True)[0]

# Prepare results
most_relevant_idx = ranked_indices[0].item()
most_relevant_doc = documents[most_relevant_idx]
most_relevant_meta = document_metadata[most_relevant_idx]

all_ranked = []
for i, idx in enumerate(ranked_indices):
    idx = idx.item()
    all_ranked.append({
        'rank': i + 1,
        'score': float(similarities[0][idx]),
        'document': documents[idx],
        'genre': document_metadata[idx]['genre'],
        'text_id': document_metadata[idx]['text_id'],
        'context': document_metadata[idx]['context']
    })

In [27]:
all_ranked

[{'rank': 1,
  'score': 0.43433254957199097,
  'document': 'title: MAG_2017 Text 2998_1 | text: the scientific enterprise @ @ @ @ @ @ @ @ @ @ far the **best system** we have for understanding our world and making it better -- which makes it all',
  'genre': 'mag_2017',
  'text_id': '2998_1',
  'context': 'the scientific enterprise @ @ @ @ @ @ @ @ @ @ far the **best system** we have for understanding our world and making it better -- which makes it all'},
 {'rank': 2,
  'score': 0.43202337622642517,
  'document': "title: NEWS_1990 Text 1390_1 | text: system , when it 's working as it should be working , really is the **best system** in the world . We have people who come from many other nations when they",
  'genre': 'news_1990',
  'text_id': '1390_1',
  'context': "system , when it 's working as it should be working , really is the **best system** in the world . We have people who come from many other nations when they"},
 {'rank': 3,
  'score': 0.4214329123497009,
  'document': 'title